# Information Theory - assignment 7
### Noam Kaplan - 200520328
### Yuval Anavim - 200496610

## Source Characterization: 
### Markovian state matrix: 

In [14]:
from mpmath import mpf, mp, fsum
import time, random, math
import numpy as np
from struct import pack
from collections import defaultdict


In [2]:
# Parameters: 
#alpha, beta, gamma, delta = mpf(0.8), mpf(0.8), mpf(0.9), mpf(0.9)
alpha, beta, gamma, delta = 0.8, 0.8, 0.9, 0.9

num_symbols = 10**5
# Set float precision: 
mp.dps = 3000

In [3]:
pr_matrix = np.asarray( [[1-alpha  , alpha/5   , 0         , 4*alpha/5 ],
                         [4*beta/5 , 1-beta    , beta/5    , 0         ],
                         [0        , 4*gamma/5 , 1-gamma   , gamma/5   ], 
                         [delta/5  , 0         , 4*delta/5 , 1-delta   ]] )

In [4]:
# Generate string of symbols as instructed. 
def generate_data(prob_mat, n=100, verbose=False): 
    symbol_list = [] # Start symbol list to empty list
    # Choose starting state randmoly: 
    prev_state = int(random.random()*4)
    for i in range(n): 
        # Choose next cell by prob
        new_state = np.random.choice([0,1,2,3], p=pr_matrix[prev_state])
        # Add simbol to list
        symbol_list.append(new_state)
        # Update prev_state
        prev_state = new_state
    if verbose: 
        print(symbol_list)
        #print(bytearray(symbol_list))
    return symbol_list

In [5]:
# Write generated data to hard drive in binary form. 
def write_binary_symbols(non_binary_symbol_list, verbose=False):
    binary_string_2write = [] # Will accumulate bytes in form ['10010011', '00000100',] 
    # Get chunks of four symbols: 
    for four_symbols in [non_binary_symbol_list[i:i+4] for i in range(0,len(non_binary_symbol_list), 4)]:
        binary_byte = [] # Will accumulate four symbols in form ['10', '01', '00']
        for symbol in four_symbols: 
            binary_byte.append('{:02b}'.format(symbol))
        binary_string_2write.append(''.join(binary_byte))
        if verbose and len(non_binary_symbol_list) < 1000: 
            print(four_symbols, binary_byte, ''.join(binary_byte), binary_string_2write)
    with open("orig.bin", "wb") as file: # Write bytes to file
        uint8_rep = [int(x, 2) for x in binary_string_2write]
        file.write(pack('<'+ 'B'*len(uint8_rep), *bytearray(uint8_rep))) 



### As instructed, We first generate a string of symbols and then write it to memory in binary form. 

In [6]:
non_binary_symbols = generate_data(pr_matrix, num_symbols) # [2, 2, 0, ..., 1, 3]
write_binary_symbols(non_binary_symbols, False)

## 1) קידוד אריתמטי
1 .יישמו קידוד אריתמטי עבור קובץ "orig "באמצעות המודל של לפלס שנלמד בכיתה, ותיצרו קובץ דחוס
ולא
(0,1,2,3)
"arithmLaplace ."שימו לב: ה-input לדוחס )בכל הסעיפים בעבודה זו( יהיו סמבולים
ביטים. לכן, עליכם לתרגם את הביטים מתוך קובץ "orig "לסמבולים. שימו לב: זמן ריצה של
האלגוריתם לא צריך לעלות על דקות ספורות.

In [7]:
def laplas_rule_prob(counter): 
    return [specific_laplas_rule_prob(counter, symbol) for symbol in sorted(counter)]

# Dynamically calculate the probability of a symbol using a counting dictionary of all symbols. 
# Return prob(symbol). 
def specific_laplas_rule_prob(counter, symbol):
    return mpf((counter[symbol]+1))/(sum((counter[key] for key in counter)) + len(counter))


In [8]:
def update_limits_original(old_low, old_high, probs, symbol):
    low  = mpf(str(old_low)) + fsum(mpf(str(probs[i])) for i in range(symbol)) * mpf((old_high-old_low))#mp.fsub
    high = mpf(str(old_low)) + fsum(mpf(str(probs[i])) for i in range(symbol+1)) * mpf((old_high-old_low))
    return low, high

In [9]:
def update_limits(old_low, old_high, probs, symbol):
    s1 =  fsum(mpf(probs[i]) for i in range(symbol))
    low  = mpf(old_low) + s1*mpf(old_high) - s1*mpf(old_low)
    s2 = fsum(mpf(probs[i]) for i in range(symbol+1))
    high = mpf(old_low) + s2*mpf(old_high) - s2*mpf(old_low)
    s2 = fsum(mpf(probs[i]) for i in range(symbol+1))
    return low, high

In [ ]:
def calc_entorpy(non_binary_symbols): 
    

In [10]:
def calc_limits(non_binary_symbols, verbose=False, fixed_probs=[]): 
    counter = {'0': 0, '1': 0, '2': 0, '3': 0}
    low, high = 0, 1
    print_indexes = [0, int(len(non_binary_symbols)/4), int(len(non_binary_symbols)/2),
                     int(3*len(non_binary_symbols)/4), len(non_binary_symbols)-1]
    prev_dps, curr_time = mp.dps, time.time()
    for index, symbol in enumerate(non_binary_symbols): # Get next symbol:
        if mp.dps*1.6 < index: 
            mp.dps += 10
            if prev_dps + 1000 == mp.dps : 
                if verbose: 
                    print('In iteration %d, new mp.dps is %d, time elpased is %.3f' %(index, mp.dps, time.time()-curr_time))
                curr_time = time.time()
                prev_dps = mp.dps
        if not fixed_probs: 
            probs = laplas_rule_prob(counter)  # Calculate each prob. 
        else:  
            probs = fixed_probs
        # If index of symbol is o, len/4, len/2, 3len/4: print probs. 
        if index in print_indexes: 
            for prob in probs: 
                print('%.5f' % prob)
            print()
        low, high = update_limits(low, high, probs, symbol) # Calculate limits.
        counter[str(symbol)] += 1          # Update counter.
        if verbose: 
            print(probs)
            print(low, high)
        if low == high: 
            print('After %d symbols, Low == High' % index)
            return low, high
    if low == high: 
        print('Low == High')
    mp.dps = 1000
    return low, high

### First We calculate low and high limits for the probability now representing the string. 
#### A printout of the symbol probabilities, after each quarter of the symbols. 

In [11]:
% time low, high = calc_limits(non_binary_symbols)

0.25000
0.25000
0.25000
0.25000

0.26528
0.26340
0.23720
0.23412

0.26494
0.26488
0.23644
0.23374

0.26551
0.26525
0.23500
0.23424

0.26408
0.26545
0.23586
0.23460

CPU times: user 1h 18min 19s, sys: 1.78 s, total: 1h 18min 21s
Wall time: 1h 18min 22s


### Size of ArithmLaplace on disk: 25kB! 

### Next we define 2 new functions to create a compressed file from these two probabilities. 
#### High_low_2_bin receives these 2 limits and caclulates the number of bits needed to represent a number inbetween, and returns a binary string with calculated length, representing a value within limits. 
#### write_binary_string receives a file name and a binary string and writes it to HD. 

In [17]:
def high_low_2_bin(high, low): 
    num_digits2write = math.ceil(mp.log(1/(high-low),2)) + 1
    hight = high
    for x in range(num_digits2write): 
        hight = mp.fmul(hight, mpf(10))
    return bin(int(hight))[2:num_digits2write+2]

def write_binary_string(file_name, binary_string): 
    byte_list = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]
    uint8_rep = [int(x, 2) for x in byte_list]
    with open(file_name+'.bin', 'wb') as t_file: 
        t_file.write(pack('<'+ 'B'*len(uint8_rep), *bytearray(uint8_rep))) 

In [18]:
compressed_binary_string = high_low_2_bin(high, low)
write_binary_string('arithmLaplace', compressed_binary_string)

### Size of ArithmAmp on disk:  25.0 kB (24,972 bytes)
Clearly, the file has hardly been compressed. This will be adressed at length in the report. 

## 2)
.תרשמו קטע קוד העובר על כל הקובץ ומחשב את ההסתברויות להופעה של כל אחד מהסימנים בקובץ.
לסט הסתברויות אלו נקרא הסתברויות ה"אמפיריות" של המקור. ענו: האם הסתברויות האמפיריות
שוות להסתברויות הסופיות מסעיף הקודם? הסבירו את התשובה. יישמו קידוד אריתמטי עבור קובץ
"orig "תחת ההנחה שהמקור מתפלג לפי הסתברויות האמפיריות שחישבתם ותיצרו קובץ דחוס
."arithmAmp"

In [19]:
def calc_amphiric_probs(symbol_list):
    counter = defaultdict(int)
    for symbol in symbol_list: 
        counter[symbol] += 1
    return [(symbol, counter[symbol]/len(symbol_list)) for symbol in sorted(counter)]

In [20]:
fixed_probs_amp = calc_amphiric_probs(non_binary_symbols)
print('Ampirical symbol probabilities: ', fixed_probs_amp)

Ampirical symbol probabilities:  [(0, 0.26409), (1, 0.26545), (2, 0.23586), (3, 0.2346)]


### As is clearly visible, these are infact the same results as in previous question. 

## We will now repeat the process of creating and writing a compressed file, this time with fixed ampirical probabilities. 

In [21]:
% time low, high = calc_limits(non_binary_symbols, fixed_probs=[ x[1] for x in fixed_probs_amp])

0.26409
0.26545
0.23586
0.23460

0.26409
0.26545
0.23586
0.23460

0.26409
0.26545
0.23586
0.23460

0.26409
0.26545
0.23586
0.23460

0.26409
0.26545
0.23586
0.23460

CPU times: user 31.4 s, sys: 13 ms, total: 31.4 s
Wall time: 31.4 s


### As before, we process the limits and save the file. 

In [23]:
compressed_binary_string = high_low_2_bin(high, low)
write_binary_string('arithmAmp', compressed_binary_string)

### Size of ArithmAmp on disk: 25.0 kB (24,969 bytes)

## 3)
יישמו קידוד אריתמטי עבור קובץ "orig "תחת ההנחה שהסמבולים הם בלתי תלויים ומתפלגים אותו
הדבר, כלומר כאשר מניחים
1
(0) (1) (2) (3)
4
, ותיצרו קובץ דחוס "arithmIid.

## We will now repeat the process of creating and writing a compressed file, this time with fixed equal probabilities. 

In [24]:
fixed_probs_iid = [(0, 0.25), (1, 0.25), (2, 0.25), (3, 0.25)]
print('Ampirical symbol probabilities: ', fixed_probs_iid)
% time low, high = calc_limits(non_binary_symbols, fixed_probs=[ x[1] for x in fixed_probs_iid])

Ampirical symbol probabilities:  [(0, 0.25), (1, 0.25), (2, 0.25), (3, 0.25)]
0.25000
0.25000
0.25000
0.25000

0.25000
0.25000
0.25000
0.25000

0.25000
0.25000
0.25000
0.25000

0.25000
0.25000
0.25000
0.25000

0.25000
0.25000
0.25000
0.25000

CPU times: user 21.9 s, sys: 5 ms, total: 21.9 s
Wall time: 21.9 s


### As before, we process the limits and save the file. 

In [25]:
compressed_binary_string = high_low_2_bin(high, low)
write_binary_string('arithmIid', compressed_binary_string)

### Size of ArithmAmp on disk: 25.0 kB (25,001 bytes)

## 4)
הפעם נשתמש במודל מתוחכם יותר, אשר יניח שהמקור שיצר את הקובץ הוא מרקובי וינסה לגלות
."arithmLaplaceMarkov" דחוס קובץ תיצרו). ,, , )
במהלך הקידוד את
נממש את המודל כך:
( 44
נחזיק מטריצה
Fji
( עבור כל המעברים אפשריים של המקור. ובדומה למודל לפלס, נספור מספר
הופעות של כל מעבר עד לסמבול הנוכחי. ועל סמך המידע שאספנו, נקבע את ההתפלגות של הסמבול
הנוכחי. כלומר,
 
X A
F
P X i X j
ji
n n


   
1
|
1
, כאשר
Fji
שהיו עד לסמבול ה-n- י,
ל- i
- זה מספר המעברים מ- j
)לא משנה למי( עד
X - זהו הגודל של הא"ב של המקור, ו- A - זה סה"כ מספר המעברים שהיו מ- j
לסמבול ה- n- י. שימו לב: זמן ריצה של האלגוריתם לא צריך לעלות על 3 דקות.

## We will now repeat the process of creating and writing a compressed file. We have a more advanced copy of the function calc_limits, called calc_limits_markov. Function or similar, only the latter has a more advanced probability model. 

In [26]:
def calc_limits_markov(non_binary_symbols, verbose=False): 
    counter = defaultdict(lambda: defaultdict(lambda: 0))
    low, high = 0, 1
    print_indexes = [0, int(len(non_binary_symbols)/4), int(len(non_binary_symbols)/2), int(3*len(non_binary_symbols)/4)]
    prev_dps, curr_time = mp.dps, time.time()
    prev_symbol = 0
    for index, symbol in enumerate(non_binary_symbols): # Get next symbol:
        probs = markov_probs(counter, prev_symbol)
        if mp.dps*1.6 < index: 
            mp.dps += 10
            if prev_dps + 1000 == mp.dps : 
                if verbose: 
                    print('In iteration %d, new mp.dps is %d, time elpased is %.3f' %(index, mp.dps, time.time()-curr_time))
                curr_time = time.time()
                prev_dps = mp.dps
        
        # If index of symbol is o, len/4, len/2, 3len/4: print probs. 
        if index in print_indexes: 
            for prob in probs: 
                print('%.10f' % prob)
            print()
        low, high = update_limits(low, high, probs, symbol) # Calculate limits.
        counter[prev_symbol][symbol] += 1          # Update counter.
        if verbose: 
            print(probs)
            print(low, high)
        if low == high: 
            print('After %d symbols, Low == High' % index)
            return low, high
        prev_symbol = symbol
    if low == high: 
        print('Low == High')
    mp.dps = 1000
    return low, high

In [27]:
def markov_probs(counter, prev): 
    return [markov_probs_specific(counter, prev, curr) for curr in range(4)]

def markov_probs_specific(counter, prev, curr):
    f_ij = counter[prev][curr]
    A = sum(counter[prev][key] for key in counter[prev])
    return mpf(1 + f_ij) / (4 + A)

In [28]:
% time low, high = calc_limits_markov(non_binary_symbols)

0.2500000000
0.2500000000
0.2500000000
0.2500000000

0.6478445659
0.1915604129
0.1604432301
0.0001517911

0.6419566694
0.1942326565
0.1637351853
0.0000754888

0.1793295771
0.0000569120
0.7186841955
0.1019293153

CPU times: user 1h 13min 25s, sys: 1.43 s, total: 1h 13min 27s
Wall time: 1h 13min 28s


### As before, we process the limits and save the file. 

In [29]:
compressed_binary_string = high_low_2_bin(high, low)
write_binary_string('arithmLaplaceMarkov', compressed_binary_string)

### Size of ArithmAmp on disk: 15.2 kB (15,178 bytes)